# Analogous-bars method
* Run the similarity-centric analogous bars method between simulated grid cells and conjunctive cells

In [1]:
using Pkg
Pkg.activate("../../env/.")

  Activating project at `~/Documents/1Projects/tracking_cycles_code/env`


In [2]:
include("../../src/analogous_bars.jl")
include("../../src/eirene_helper.jl")

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/iYDwd/src/backends/hdf5.jl", 37, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010193cb1f, Ptr{Nothing} @0x00000001019e63cf, Ptr{Nothing} @0x00000001019e757b, Ptr{Nothing} @0x00000001019e5ae6, Ptr{Nothing} @0x00000001019e60dc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x0000000101a0115f, Ptr{Nothing} @0x0000000101a012f5, Ptr{Nothing} @0x0000000101a00f02, Ptr{Nothing} @0x0000000101a00f02, Ptr{Nothing} @0x0000000101a021d5, Ptr{Nothing} @0x0000000118be47da, Ptr{Nothing} @0x00000001019ce160, Ptr{Nothing} @0x0000000118c23f45, Ptr{Nothing} @0x00000001187afc85, Ptr{Nothing} @0x00000001019ce160, Ptr{Nothing} @0x00000001019daf4a, Ptr{Nothing} @0x0000000102f06baa, Ptr{Nothing} @0x00000001019ce160, Ptr{Nothing} @0x00000001019e759f, Ptr{Nothing} @0x00000001019e5ae6, Ptr{Nothing} @0x000000

Main.helper

In [6]:
using .analogous_bars
using .helper
using DelimitedFiles
using Printf
using HDF5
using Eirene
using Plots
using JLD

# 1. Load distance matrices


In [12]:
directory = "simulated_data/"

n_grid = 200
n_conj = 800

D_grid = h5read(directory * "grid_distance.h5", "distance")
D_grid = vector_to_symmetric_matrix(D_grid, n_grid)

D_conj = h5read(directory * "conj_distance.h5", "distance")
D_conj = vector_to_symmetric_matrix(D_conj, n_conj)

D_grid_conj = h5read(directory * "conj_grid_rate_distance.h5", "distance")
D_conj_grid = Array(transpose(D_grid_conj));

# 2. Compute persistence

In [14]:
dim = 1

# VR persistence
VR_grid = eirene(D_grid, record = "all", maxdim = dim)
VR_conj = eirene(D_conj, record = "all", maxdim = dim);

In [17]:
# Witness persistence
W_grid_conj = compute_Witness_persistence(D_grid_conj, maxdim = dim)
W_conj_grid = compute_Witness_persistence(D_conj_grid, maxdim = dim);

In [ ]:
# get barcodes
barcode_VR_grid = barcode(VR_grid, dim = 1)
barcode_VR_conj = barcode(VR_conj, dim = 1)
barcode_W_grid_conj = barcode(W_grid_conj["eirene_output"], dim = 1)
barcode_W_conj_grid = barcode(W_conj_grid["eirene_output"], dim = 1)

In [50]:
# compute persistene cutoff scores
grid_idx, grid_cutoff = select_persistent_intervals_IQR(barcode_VR_grid)
W_idx, W_cutoff = select_persistent_intervals_IQR(barcode_W_grid_conj)
conj_idx, conj_cutoff = select_persistent_intervals_IQR(barcode_VR_conj);

([58, 59], 0.30071595555918307)

In [64]:
# plot persistence diagrams
p1 = plot_PD(barcode_VR_grid, title = "VR(grid)", highlight = grid_idx, cutoff = grid_cutoff)
p2 = plot_PD(barcode_W_grid_conj, title = "W(grid, conj)", highlight = W_idx, cutoff = W_cutoff)
p3 = plot_PD(barcode_VR_conj, title = "VR(conj)", highlight = conj_idx, cutoff = conj_cutoff)
plot(p1, p2, p3, layout = grid(1,3), size = (500, 200))

<!DOCTYPE html>
 
 
 Plots.jl

# 3. Run baseline similarity-centric analogous bars


In [21]:
selected = [137, 136]

2-element Array{Int64,1}:
 137
 136

In [22]:
analogous_grid, analogous_conj = ext_var.run_baseline_similarity_analogous(VR_P = VR_grid,
                                                                    D_P = D_grid,
                                                                    VR_Q = VR_conj,
                                                                    D_Q = D_conj,
                                                                    W_PQ = W_grid_conj,
                                                                    W_PQ_bars = selected,
                                                                    dim = 1);


error: keyword argument <rows> contains an integer greater than the rank of the complex in dimension <dim-1>

error: keyword argument <rows> contains an integer greater than the rank of the complex in dimension <dim-1>

error: keyword argument <rows> contains an integer greater than the rank of the complex in dimension <dim-1>

error: keyword argument <rows> contains an integer greater than the rank of the complex in dimension <dim-1>


## Plot analogous pairs

### Analogous pair via witness bar 136

In [68]:
# plot persistence diagrams
witness_bar = 136
p1 = plot_PD(barcode_VR_grid, title = "VR(grid)", highlight = analogous_grid[witness_bar]["baseline_bar_extension"], cutoff = grid_cutoff)
p2 = plot_PD(barcode_W_grid_conj, title = "W(grid, conj)", highlight = [witness_bar], cutoff = W_cutoff)
p3 = plot_PD(barcode_VR_conj, title = "VR(conj)", highlight = analogous_conj[witness_bar]["baseline_bar_extension"], cutoff = conj_cutoff)
plot(p1, p2, p3, layout = grid(1,3), size = (500, 200))

<!DOCTYPE html>
 
 
 Plots.jl

### Analogous pair via Witness bar 137

In [69]:
# plot persistence diagrams
witness_bar = 137
p1 = plot_PD(barcode_VR_grid, title = "VR(grid)", highlight = analogous_grid[witness_bar]["baseline_bar_extension"], cutoff = grid_cutoff)
p2 = plot_PD(barcode_W_grid_conj, title = "W(grid, conj)", highlight = [witness_bar], cutoff = W_cutoff)
p3 = plot_PD(barcode_VR_conj, title = "VR(conj)", highlight = analogous_conj[witness_bar]["baseline_bar_extension"], cutoff = conj_cutoff)
plot(p1, p2, p3, layout = grid(1,3), size = (500, 200))

<!DOCTYPE html>
 
 
 Plots.jl

In [53]:
# gather barcodes
barcodes = Dict()
barcodes["VR_grid"] = barcode_VR_grid
barcodes["VR_conj"] = barcode_VR_conj
barcodes["W_grid_conj"] = barcode_W_grid_conj

# gather the persistence cutoff and selected bars
persistence_cutoff = Dict()
persistence_cutoff["grid_selected"] = grid_idx
persistence_cutoff["grid_cutoff"] = grid_cutoff
persistence_cutoff["conj_selected"] = conj_idx
persistence_cutoff["conj_cutoff"] = conj_cutoff
persistence_cutoff["W_grid_conj_selected"] = W_idx
persistence_cutoff["W_grid_conj_cutoff"] = W_cutoff;

# gather analogous pairs
analogous_pair = Dict()

for i in selected
    pair_i = Dict()
    pair_i["order"] = ["grid", "conjunctive"]
    pair_i["baseline_bar_ext"] = [analogous_grid[i]["baseline_bar_extension"], analogous_conj[i]["baseline_bar_extension"]]
    pair_i["epsilon_0"] = [analogous_grid[i]["epsilon_0"], analogous_conj[i]["epsilon_0"]]
    pair_i["min_epsilon_with_boundary"] = [analogous_grid[i]["min_epsilon_with_boundary"], analogous_conj[i]["min_epsilon_with_boundary"]]
    analogous_pair[i] = pair_i
end

# save output
"""
save("simulated_data/grid_conj_analogous_outputs.jld2", 
    "barcodes", barcodes,
    "persistence_cutoff", persistence_cutoff,
    "analogous_pair", analogous_pair)
"""


In [57]:
# save Witness persistence info
"""
save("simulated_data/grid_conj_W_persistence.jld2", 
   "eirene_output", W_grid_conj["eirene_output"],
    "index2simplex", W_grid_conj["index2simplex"],
    "W_vertex_to_defualt_vertex", W_grid_conj["W_vertex_to_default_vertex"])

save("simulated_data/conj_grid_W_persistence.jld2", 
    "eirene_output", W_conj_grid["eirene_output"],
    "index2simplex", W_conj_grid["index2simplex"],
    "W_vertex_to_defualt_vertex", W_conj_grid["W_vertex_to_default_vertex"])
"""